In [1]:
import pandas as pd
import os

In [2]:
"""Functions Related to Tranfering Data to Excel Sheet"""
def writeDataframe6(wbpath,df,sheetname,dfcols,xlcols,indexcol):    
    wb=opxl.load_workbook(wbpath)
    wsheet=wb[sheetname]    
    for index,row in df.iterrows():        
        myrow=int(row[indexcol])
        for idf,ixl in zip(dfcols,xlcols):
            wsheet.cell(row=myrow,column=ixl).value=row[idf]               
    wb.save(wbpath)
def saveDataFrame(dataframes,filepath,names):
    writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
    for df,sname in zip(dataframes,names):
        df.to_excel(writer,sheet_name=sname)   
    writer.save()  

In [3]:
"""Packagewise and Structurewise Summation"""
def packageWiseSum(package_df,projection_df):
    plist=list(package_df['Package_No'])
    for index,row in projection_df.iterrows():
        pname=row['Package']
        row_index=plist.index(pname)
        package_df.iloc[row_index,1]=package_df.iloc[row_index,1]+row['VDW_30_06_2020']
        package_df.iloc[row_index,2]=package_df.iloc[row_index,2]+row['VDW_30_06_2021']
    for index,row in package_df.iterrows():
        package_df.iloc[index,4]= package_df.iloc[index,2]- package_df.iloc[index,3]
    return package_df
def structureWiseSum(structure_df,projection_df):
    plist=list(structure_df['Code'])
    for index,row in projection_df.iterrows():
        pname=row['Structure Code']
        row_index=plist.index(pname)
        structure_df.iloc[row_index,3]=structure_df.iloc[row_index,3]+row['VDW_30_06_2020']
        structure_df.iloc[row_index,4]=structure_df.iloc[row_index,4]+row['VDW_30_06_2021']
    for index,row in structure_df.iterrows():
        structure_df.iloc[index,6]= structure_df.iloc[index,4]- structure_df.iloc[index,5]
    return structure_df

def dppWiseSum(structure_df,dpp_df):    
    dpp_item=list(dpp_df['DPP_Item']) #DPP_Item
    for index,row in structure_df.iterrows():
        #item_name=row[]
        pname=row['DPP_Item']
        row_index= dpp_item.index(pname)
        dpp_df.iloc[row_index,1]= dpp_df.iloc[row_index,1]+row['VWD_30_06_2020']
        dpp_df.iloc[row_index,2]=dpp_df.iloc[row_index,2]+row['VWD_30_06_2021']
    for index,row in dpp_df.iterrows():
        dpp_df.iloc[index,4]= dpp_df.iloc[index,2]- dpp_df.iloc[index,3]
        for col in range(1,5):
             dpp_df.iloc[index,col]=round(dpp_df.iloc[index,col],2)
       
       
    return dpp_df



In [4]:
def buildPackageWisePaymentMatrix(package_df):
    #Package_No Payment_30_06_2020
    package=list(package_df['Package_No'])
    paid=list(package_df['Payment_30_06_2020'])
    zero_list=[0 for x in package]
    data={'Package_No':package,'VWD_30_06_2020':zero_list,'VWD_30_06_2021':zero_list,
          'Payment_30_06_2020':paid,'Payment_30_06_2021':zero_list}
    package_payment_df=pd.DataFrame(data)
    return package_payment_df
def buildStructureWisePaymentMatrix(structure_df):
    #DPP Item	Initial	Code
    structure_name=list(structure_df['Items'])
    structure_code=list(structure_df['Code'])
    DPP_Item_code=list(structure_df['DPP_Item'])  
    payment=list(structure_df['Payment_30_06_2020'])
    #Package_No Payment_30_06_2020    
    zero_list=[0 for x in structure_name]
    data={'Items':structure_name,'Code':structure_code ,'DPP_Item':DPP_Item_code,'VWD_30_06_2020':zero_list,'VWD_30_06_2021':zero_list,
          'Payment_30_06_2020': payment,'Payment_30_06_2021':zero_list}
    structure_payment_df=pd.DataFrame(data)
    return structure_payment_df
    
def buildVWDMatrix(projection_df):
    #Component	Unit	Qnty	Cost	Present_Progress	Projected_progress
    #Package
    #Structure Code
    stc=list(projection_df['Structure Code'])
    packages=list(projection_df['Package'])
    components=list(projection_df['Component'])
    units=list(projection_df['Unit'])
    quantity=list(projection_df['Qnty'])
    cost=list(projection_df['Cost'])
    present_prog=list(projection_df['Present_Progress'])
    projected_prog=list(projection_df['Projected_progress'])
    zero_list=[0 for x in components ]
    data={'Package':packages,'Structure Code':stc,'Component':components,'Unit':units,'Qnty':quantity,'Cost':cost,
          'Present_Progress':present_prog,'Projected_progress':projected_prog,
          'VDW_30_06_2020': zero_list,'VDW_30_06_2021':zero_list,'Work_Done_20_21':zero_list,'Payment_30_06_2020': zero_list,
          'Payment_30_06_2020':zero_list}
    vwd_mat=pd.DataFrame(data)
    for index,row in vwd_mat.iterrows():
        vwd_mat.iloc[index,8]=vwd_mat.iloc[index,5]*vwd_mat.iloc[index,6]
        vwd_mat.iloc[index,9]=vwd_mat.iloc[index,5]*vwd_mat.iloc[index,7]
        vwd_mat.iloc[index,10]=vwd_mat.iloc[index,9]-vwd_mat.iloc[index,8]
    return vwd_mat
    


In [5]:
myfolder=r'E:\Website_24_11_2020\cmis6\cmis6\Civilworks cost\RADP Preparations' #Home Computer
#myfolder=r'F:\website\cmis6\Civilworks cost\RADP Preparations' #Office Computer
input_file_name=os.path.join(myfolder,'Projecttion_input.xlsx')
out_file_name=os.path.join(myfolder,'Projecttion_output.xlsx')
myframes=[]
mynames=[]
sheetName="Structure_Type"
structure_type_df=pd.read_excel(input_file_name,sheet_name=sheetName)
structure_type_df.fillna(0,inplace=True)
structure_type_df
myframes.append(structure_type_df)
mynames.append("Structure_Type")
"""Projection Info"""
#sheetName="Kishoregnj"
sheetName="Kishoregnj"
projection_info_df=pd.read_excel(input_file_name,sheet_name=sheetName)
projection_info_df.fillna(0,inplace=True)
projection_info_df
myframes.append(projection_info_df)
mynames.append("Projection_Info")
sheetName="Packages"
package_name_df=pd.read_excel(input_file_name,sheet_name=sheetName)
package_name_df.fillna(0,inplace=True)
myframes.append(package_name_df)
mynames.append("Package_Name")
vwd_df=buildVWDMatrix(projection_info_df)
myframes.append(vwd_df)
mynames.append("Value_of_Work_Done")
package_payment_df=buildPackageWisePaymentMatrix(package_name_df)
package_payment_df=packageWiseSum(package_payment_df,vwd_df)
myframes.append(package_payment_df)
mynames.append("Package_wise_payment")
"""Building Structurewise Cost"""
structure_payment_df=buildStructureWisePaymentMatrix(structure_type_df)
structure_payment_df=structureWiseSum(structure_payment_df,vwd_df)
myframes.append(structure_payment_df)
mynames.append("Structure_Wise_payment")
"""Building Structurewise Cost"""
sheetName="DPP_Item"
dpp_item_df=pd.read_excel(input_file_name,sheet_name=sheetName)
dpp_item_df.fillna(0,inplace=True)
dpp_item_df=dppWiseSum(structure_payment_df,dpp_item_df)
myframes.append(dpp_item_df)
mynames.append("DPP_Wise_payment")
saveDataFrame(myframes,out_file_name,mynames)



In [6]:
projection_info_df

,Package Name,Component,Unit,Qnty,Cost,Present_Progress,Projected_progress,Package,Haor Type,Structure Code,Earth Volume,Hoar Code,District Code
0,BWDB/Kish/HFMLIP/ PW-01,Betal Doba from KM 1.315 to KM 3.466 =2.151 KM,Km,2.151,40.940000,1.0,1.0,KISH/PW-01,Rehab,8,0.0,R-2,KISH
1,BWDB/Kish/HFMLIP/ PW-01,Alalia-bahadia Khal from KM 1.325 to KM 2.545=...,Km,1.220,35.880000,1.0,1.0,KISH/PW-01,Rehab,8,0.0,R-1,KISH
2,BWDB/Kish/HFMLIP/ PW-01,katakhali khal from KM 0.000 to KM 1.550=1.550 KM,Km,1.550,19.180000,0.0,1.0,KISH/PW-01,Rehab,8,0.0,R-3,KISH
3,BWDB/Kish/HFMLIP/ PW-01,Ganakkhali Khal from KM 0.000 to KM 5.625=5.62...,Km,5.625,309.800000,0.0,1.0,KISH/PW-01,Rehab,8,0.0,R-3,KISH
4,BWDB/Kish/HFMLIP/ PW-01,kali Khal from KM 0.00 to KM 1.000=1.000 KM,Km,1.000,45.950000,0.0,1.0,KISH/PW-01,Rehab,8,0.0,R-3,KISH
...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,BWDB/Sunam/HFMLIP/PW-06 Dharmapasha Rui Beel (...,Kakura Khal,km,1.305,32.572846,1.0,1.0,SUNM/PW-06,New,7,0.0,N-12,SUNM
284,BWDB/Sunam/HFMLIP/PW-06 Dharmapasha Rui Beel (...,Dharam Beel Khal,km,1.105,37.708401,1.0,1.0,SUNM/PW-06,New,7,0.0,N-12,SUNM
285,BWDB/Sunam/HFMLIP/PW-06 Dharmapasha Rui Beel (...,Bararia Khal,km,2.110,14.600076,1.0,1.0,SUNM/PW-06,New,7,0.0,N-12,SUNM
286,BWDB/Sunam/HFMLIP/PW-06 Dharmapasha Rui Beel (...,Const. of Box Drainage Outlet-1,Nos,1.000,45.130000,0.0,1.0,SUNM/PW-06,New,4,0.0,N-12,SUNM


In [7]:
dpp_item_df

,DPP_Item,VWD_30_06_2020,VWD_30_06_2021,Payment_30_06_2020,Payment_30_06_2021
0,Inlet,712.75,975.77,291.81,683.96
1,Reg_Rehab_Rehab,370.09,434.36,0.00,434.36
2,Reg_CW_Box,11527.27,17395.34,6503.97,10891.37
3,Khal_Riv_New,6994.39,8557.35,2546.54,6010.81
4,Khal_Riv_Rehab,2514.79,3367.48,51.09,3316.39
5,Full_Emb_Rehab,1593.75,1795.55,0.00,1795.55
6,Sub_Emb_Rehab,1624.37,1752.83,0.00,1752.83
7,Sub_Emb_Const,8844.85,10909.67,5833.04,5076.63
8,Reg_Rehab_New,19.43,139.99,0.00,139.99
9,Thr_Fl_Cost,0.00,0.00,0.00,0.00


In [8]:
sheetName="Packagewise_sum"
contract_value_df=pd.read_excel(input_file_name,sheet_name=sheetName)
contract_value_df.fillna(0,inplace=True)
contract_value_df
#Cost	Present_Progress	Projected_progress	Package	Haor Type	Structure Code
package_list=list(contract_value_df['Package_No'])
col_names=list(contract_value_df.columns)
for index,row in projection_info_df.iterrows():
    cost=row['Cost']
    stc_code=row['Structure Code']
    pno=row['Package']
    r=package_list.index(pno)
    c=col_names.index(stc_code)
    contract_value_df.iloc[r,c]=contract_value_df.iloc[r,c]+ cost
    print("cost={} stc_code={} package_no={} row={},col={}".format(cost,stc_code,pno,r,c))
for index,row in contract_value_df.iterrows():
    total=0
    for i in range(1,17):
        total=total+contract_value_df.iloc[index,i]
        print(contract_value_df.iloc[index,i])
    contract_value_df.iloc[index,17]=total
structure_wise_distribution_df=contract_value_df.copy(deep=True)
for index,row in structure_wise_distribution_df.iterrows():
    total=structure_wise_distribution_df.iloc[index,17]
    for i in range(1,17):
        print(i)
        val=float(structure_wise_distribution_df.iloc[index,i])
        df=round(val/total,4)
        structure_wise_distribution_df.iloc[index,i]=df
        
        #df=val/total
        #structure_wise_distribution_df.iloc[index,i]=round(structure_wise_distribution_df/total,4)
for index,row in structure_wise_distribution_df.iterrows():
    total=0
    for i in range(1,17):
        total=total+structure_wise_distribution_df.iloc[index,i]
        #print(i)
        #val=float(structure_wise_distribution_df.iloc[index,i])
        #df=round(val/total)
        #structure_wise_distribution_df.iloc[index,i]=df
    structure_wise_distribution_df.iloc[index,17]=total
        
        #df=val/total
        #structure_wise_distribution_df.iloc[index,i]=round(structure_wise_distribution_df/total,4)   
#for index,row in contract_value_df.iterrows():
    #total=0
    #for i in range(1,17):
        #total=total+contract_value_df[index,i]
    #contract_value_df.iloc[index,17]=total
        


cost=40.94 stc_code=8 package_no=KISH/PW-01 row=0,col=8
cost=35.88 stc_code=8 package_no=KISH/PW-01 row=0,col=8
cost=19.18 stc_code=8 package_no=KISH/PW-01 row=0,col=8
cost=309.8 stc_code=8 package_no=KISH/PW-01 row=0,col=8
cost=45.95 stc_code=8 package_no=KISH/PW-01 row=0,col=8
cost=44.79 stc_code=2 package_no=KISH/PW-01 row=0,col=2
cost=155.51144882000003 stc_code=3 package_no=KISH/PW-02 row=1,col=3
cost=31.95760891 stc_code=7 package_no=KISH/PW-02 row=1,col=7
cost=93.43512043 stc_code=7 package_no=KISH/PW-02 row=1,col=7
cost=52.1721892 stc_code=7 package_no=KISH/PW-02 row=1,col=7
cost=16.20103913 stc_code=7 package_no=KISH/PW-02 row=1,col=7
cost=68.85988 stc_code=7 package_no=KISH/PW-02 row=1,col=7
cost=14.810948759999999 stc_code=11 package_no=KISH/PW-02 row=1,col=11
cost=155.5114 stc_code=3 package_no=KISH/PW-03 row=2,col=3
cost=247.1255 stc_code=11 package_no=KISH/PW-03 row=2,col=11
cost=286.8206 stc_code=11 package_no=KISH/PW-03 row=2,col=11
cost=97.95786808717001 stc_code=3 pac

cost=62.22 stc_code=4 package_no=NETR/PW-08 row=33,col=4
cost=12.18 stc_code=1 package_no=NETR/PW-08 row=33,col=1
cost=12.18 stc_code=1 package_no=NETR/PW-08 row=33,col=1
cost=12.18 stc_code=1 package_no=NETR/PW-08 row=33,col=1
cost=151.385 stc_code=10 package_no=HOBI/PW-01 row=40,col=10
cost=151.385 stc_code=10 package_no=HOBI/PW-01 row=40,col=10
cost=732.6 stc_code=8 package_no=HOBI/PW-01 row=40,col=8
cost=99.9 stc_code=8 package_no=HOBI/PW-01 row=40,col=8
cost=208.0 stc_code=3 package_no=HOBI/PW-01 row=40,col=3
cost=178.13 stc_code=5 package_no=HOBI/PW-01 row=40,col=5
cost=274.56 stc_code=10 package_no=HOBI/PW-02 row=41,col=10
cost=225.46 stc_code=10 package_no=HOBI/PW-02 row=41,col=10
cost=318.42 stc_code=10 package_no=HOBI/PW-02 row=41,col=10
cost=149.54 stc_code=10 package_no=HOBI/PW-02 row=41,col=10
cost=132.8661023 stc_code=8 package_no=HOBI/PW-02 row=41,col=8
cost=53.04748093 stc_code=8 package_no=HOBI/PW-02 row=41,col=8
cost=29.97498828 stc_code=8 package_no=HOBI/PW-02 row=41

7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
1

In [9]:
myframes.append(contract_value_df)
mynames.append("Package_wise_str_value")
myframes.append(structure_wise_distribution_df)
mynames.append("Package_wise_str_df")
saveDataFrame(myframes,out_file_name,mynames)

In [10]:
structure_wise_distribution_df

,Package_No,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,Total
0,KISH/PW-01,0.0000,0.0902,0.0000,0.0000,0.0000,0.0,0.0000,0.9098,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,1.0000
1,KISH/PW-02,0.0000,0.0000,0.3592,0.0000,0.0000,0.0,0.6066,0.0000,0.0000,0.0000,0.0342,0.0000,0.0,0.0,0.0,0.0,1.0000
2,KISH/PW-03,0.0000,0.0000,0.2256,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.7744,0.0000,0.0,0.0,0.0,0.0,1.0000
3,KISH/PW-04,0.0000,0.0000,0.6385,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.3615,0.0000,0.0,0.0,0.0,0.0,1.0000
4,KISH/PW-05,0.0000,0.0000,0.2478,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.7522,0.0000,0.0,0.0,0.0,0.0,1.0000
5,KISH/PW-06,0.0000,0.0000,0.2942,0.0000,0.0000,0.0,0.7058,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,1.0000
6,KISH/PW-07,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.9138,0.0000,0.0000,0.0000,0.0862,0.0000,0.0,0.0,0.0,0.0,1.0000
7,KISH/PW-09,0.0000,0.0000,0.5555,0.0000,0.4445,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,1.0000
8,KISH/PW-10,0.0000,0.0000,0.3009,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.6991,0.0000,0.0,0.0,0.0,0.0,1.0000
9,KISH/PW-11,0.0000,0.0000,0.3299,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.6701,0.0000,0.0,0.0,0.0,0.0,1.0000
